# Linear Regression implementation utilizing Pytorch

In [1]:
!pip install --upgrade torch torchvision torchaudio scikit-learn numpy matplotlib
import numpy as np
import torch
import matplotlib.pyplot as plt

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable


In [2]:
print(torch.__version__)

2.5.1+cu124


In [36]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaler = StandardScaler()
housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

X = torch.tensor(housing_data_plus_bias, dtype=torch.float32)
y = torch.tensor(housing.target.reshape(-1, 1), dtype=torch.float32)

if torch.cuda.is_available():
    print("GPU is available. Moving X, and y to GPU")
    X = X.to(device='cuda')
    y = y.to(device='cuda')

GPU is available. Moving X, and y to GPU


In [40]:
# Define hyperparameters
n_epochs = 10000
learning_rate = 0.01

# Zero the gradients
theta = 2 * (torch.rand(X.shape[1], 1, dtype=torch.float32) - 0.5)
# theta.grad.zero_()
if torch.cuda.is_available():
    print("GPU is available. to GPU")
    theta = theta.to(device='cuda')

theta.requires_grad_(True)

for epoch in range(n_epochs + 1):
    # Compute predictions
    y_pred = X @ theta  # Matrix multiplication

    # Compute error and MSE loss
    error = y_pred - y
    mse = (error ** 2).mean()

    # Compute gradients
    mse.backward()

    # Update theta using gradient descent
    with torch.no_grad():
        theta -= learning_rate * theta.grad

    # Zero the gradients
    theta.grad.zero_()

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, MSE = {mse.item()}")

GPU is available. to GPU
Epoch 0, MSE = 8.881109237670898
Epoch 100, MSE = 4.916640281677246
Epoch 200, MSE = 4.81827974319458
Epoch 300, MSE = 4.811000347137451
Epoch 400, MSE = 4.808780193328857
Epoch 500, MSE = 4.807347297668457
Epoch 600, MSE = 4.806306838989258
Epoch 700, MSE = 4.805540084838867
Epoch 800, MSE = 4.804973602294922
Epoch 900, MSE = 4.8045525550842285
Epoch 1000, MSE = 4.804239749908447
Epoch 1100, MSE = 4.8040056228637695
Epoch 1200, MSE = 4.803830146789551
Epoch 1300, MSE = 4.8036980628967285
Epoch 1400, MSE = 4.803597927093506
Epoch 1500, MSE = 4.803521156311035
Epoch 1600, MSE = 4.803463459014893
Epoch 1700, MSE = 4.8034186363220215
Epoch 1800, MSE = 4.803383827209473
Epoch 1900, MSE = 4.8033576011657715
Epoch 2000, MSE = 4.803336143493652
Epoch 2100, MSE = 4.803319931030273
Epoch 2200, MSE = 4.803306579589844
Epoch 2300, MSE = 4.80329704284668
Epoch 2400, MSE = 4.803288459777832
Epoch 2500, MSE = 4.803282260894775
Epoch 2600, MSE = 4.803276538848877
Epoch 2700, 

In [42]:
# Define hyperparameters
n_epochs = 10000
learning_rate = 0.01

# Initialize theta as a trainable parameter
theta = torch.randn((X.shape[1], 1), dtype=torch.float32, device='cuda', requires_grad=True)

# Define optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD([theta], lr=learning_rate)

# Training loop
for epoch in range(n_epochs + 1):
    # Compute predictions
    y_pred = X @ theta  # Matrix multiplication

    # Compute loss (Mean Squared Error)
    mse = (y_pred - y).pow(2).mean()

    # Zero the gradients before backpropagation
    optimizer.zero_grad()

    # Compute gradients via backpropagation
    mse.backward()

    # Update theta using optimizer
    optimizer.step()

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, MSE = {mse.item()}")

# Final optimized theta
best_theta = theta.detach().cpu().numpy()
print("Optimized Theta:", best_theta)

Epoch 0, MSE = 17.94706916809082
Epoch 100, MSE = 5.059835433959961
Epoch 200, MSE = 4.846272945404053
Epoch 300, MSE = 4.8298420906066895
Epoch 400, MSE = 4.823822021484375
Epoch 500, MSE = 4.819561004638672
Epoch 600, MSE = 4.8162641525268555
Epoch 700, MSE = 4.813679218292236
Epoch 800, MSE = 4.811641693115234
Epoch 900, MSE = 4.8100266456604
Epoch 1000, MSE = 4.808741569519043
Epoch 1100, MSE = 4.807713985443115
Epoch 1200, MSE = 4.806889057159424
Epoch 1300, MSE = 4.806224346160889
Epoch 1400, MSE = 4.805686950683594
Epoch 1500, MSE = 4.805250644683838
Epoch 1600, MSE = 4.804895877838135
Epoch 1700, MSE = 4.804605960845947
Epoch 1800, MSE = 4.8043694496154785
Epoch 1900, MSE = 4.804174900054932
Epoch 2000, MSE = 4.804015159606934
Epoch 2100, MSE = 4.803884029388428
Epoch 2200, MSE = 4.803775787353516
Epoch 2300, MSE = 4.803686618804932
Epoch 2400, MSE = 4.80361270904541
Epoch 2500, MSE = 4.803552150726318
Epoch 2600, MSE = 4.803501129150391
Epoch 2700, MSE = 4.803459167480469
Epoc

In [49]:
from torch.utils.data import TensorDataset, DataLoader

device='cuda'

# Define hyperparameters
n_epochs = 10000
learning_rate = 0.001
batch_size = 128  # Define mini-batch size

# Create mini-batch DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize theta as a trainable parameter
theta = torch.randn((X.shape[1], 1), dtype=torch.float32, device=device, requires_grad=True)

# Define optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD([theta], lr=learning_rate)

# Training loop
for epoch in range(n_epochs + 1):
    for batch_X, batch_y in dataloader:  # Iterate over mini-batches
        # Compute predictions
        y_pred = batch_X @ theta  # Matrix multiplication

        # Compute loss (Mean Squared Error)
        mse = (y_pred - batch_y).pow(2).mean()

        # Zero the gradients before backpropagation
        optimizer.zero_grad()

        # Compute gradients via backpropagation
        mse.backward()

        # Update theta using optimizer
        optimizer.step()

    # Print loss every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, MSE = {mse.item()}")

# Final optimized theta
best_theta = theta.detach().cpu().numpy()
print("Optimized Theta:", best_theta)

Epoch 0, MSE = 6.470011234283447
Epoch 100, MSE = 5.432498931884766
Epoch 200, MSE = 4.13297700881958
Epoch 300, MSE = 3.7448270320892334
Epoch 400, MSE = 4.711345672607422
Epoch 500, MSE = 4.623612403869629
Epoch 600, MSE = 4.4607744216918945
Epoch 700, MSE = 5.024688720703125
Epoch 800, MSE = 5.540892124176025
Epoch 900, MSE = 3.9796359539031982
Epoch 1000, MSE = 5.572781562805176
Epoch 1100, MSE = 3.4575204849243164
Epoch 1200, MSE = 4.971525192260742
Epoch 1300, MSE = 6.160943984985352
Epoch 1400, MSE = 4.750181674957275
Epoch 1500, MSE = 5.0050554275512695
Epoch 1600, MSE = 7.139811038970947
Epoch 1700, MSE = 3.919447422027588
Epoch 1800, MSE = 6.097949028015137
Epoch 1900, MSE = 4.586579322814941
Epoch 2000, MSE = 4.15958309173584
Epoch 2100, MSE = 4.3302741050720215
Epoch 2200, MSE = 5.279644012451172
Epoch 2300, MSE = 4.347583770751953
Epoch 2400, MSE = 5.252175331115723
Epoch 2500, MSE = 4.770808219909668
Epoch 2600, MSE = 4.795855522155762
Epoch 2700, MSE = 5.189198970794678
